In [ ]:
# Imports
import pandas as pd
import numpy as np
import duckdb

In [ ]:
# Load data
dnc_df = duckdb.read_parquet('output/DNC/dnc_list_final.parquet').df()
raw_df = duckdb.read_csv('dataset/RAW/JAS233_JBT.csv', normalize_names=True).df()

In [ ]:
# Convert phone to int
raw_df['work_phone'] = raw_df['work_phone'].astype(np.int64)

In [ ]:
# Checks if Phone from RAW file is in DNC list
is_dnc = duckdb.sql("""
    SELECT DISTINCT a.*,
        CASE
            WHEN b.value IS NOT NULL THEN 'Yes' ELSE 'No'
        END AS is_dnc
    FROM raw_df a
    LEFT JOIN dnc_df b on a.work_phone = b.value
""").df()

is_dnc.tail(3)

In [ ]:
# Export to Excel
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
                  
    COPY is_dnc TO 'output/jas233_jbt_cleaned_copy.xlsx' WITH (FORMAT GDAL, DRIVER 'xlsx')
""")